# Import Packages

In [1]:
from flask import *
import csv
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from textblob import TextBlob
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib import cm
import shutil
from Sentiment_Analysis_sdm import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\altha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\altha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\altha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\altha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Index(['Brand', 'Review'], dtype='object')


# Flask Application 

In [23]:

app = Flask(__name__)


@app.route('/')
def hello():
    return render_template('Home.html')
@app.route("/Dashboard", methods=['POST','GET'])
def Dashboard():
    print("in dashboard function")
    return render_template("Dashboard.html")

@app.route("/sdm", methods=['POST','GET'])
def sdm():
    
    from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
    from msrest.authentication import ApiKeyCredentials
    from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
    from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry

    ENDPOINT = "XXXXXXXXXXXXXXXXX"
    
    training_key = "XXXXXXXXXXXXXXXx"
    prediction_key = "XXXXXXXXXXXXXXXXX"
    prediction_resource_id = "XXXXXXXXXXXXX"
    
    publish_iteration_name = "classifyModel"

    credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
    trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
    
    # Now there is a trained endpoint that can be used to make a prediction
    prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
    predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

    project = trainer.get_project("611bc936-654c-4a48-82e9-703ed40708a7")
    
    if request.method=='POST':
        print("here")
        try:
            file = request.form["file"]
            print(file)
        except Exception as e:
            print(e)
            pass
    
    with open(file, "rb") as image_contents:
        results = predictor.classify_image(
            project.id, publish_iteration_name, image_contents.read())
        
        print("\t" + results.predictions[0].tag_name +
              ": {0:.2f}%".format(results.predictions[0].probability * 100))
        
        predicted_class = results.predictions[0].tag_name
        predicted_prob = results.predictions[0].probability * 100
    path_to_file = "C:/Users/altha/althaf_docs/Sem 2/Social Data Mining Techniques/Final Assignment/"+file
    print(path_to_file)
    print("in sdm function")
    
    
    shutil.copy(file, r"C:\Users\altha\althaf_docs\Sem 2\Social Data Mining Techniques\Final Assignment\static")
    
    #sentiment analysis
    df = pd.read_csv("Sentiment_Analysis.csv",encoding="cp1252")
    df = df.dropna()
    
    if predicted_class == "Adidas":
        Desc = "Adidas AG is a multinational corporation, founded and headquartered in Herzogenaurach, Germany, that designs and manufactures shoes, clothing and accessories. It is the largest sportswear manufacturer in Europe, and the second largest in the world, after Nike. "
        result_df = df[df['Brand']=="Adidas"][['Review','Sentiments','Polarity']]
        result_df = result_df.reset_index()[['Review','Sentiments','Polarity']]
        image = "Adidas.png"
      
    elif predicted_class =="Nike":
        Desc = "Nike, Inc. is an American multinational corporation that is engaged in the design, development, manufacturing, and worldwide marketing and sales of footwear, apparel, equipment, accessories, and services. The company is headquartered near Beaverton, Oregon, in the Portland metropolitan area."
        result_df = df[df['Brand']=="Nike"][['Review','Sentiments','Polarity']]
        result_df = result_df.reset_index()[['Review','Sentiments','Polarity']]
        image = "Nike.png"
        
    else:
        Desc = "Puma SE, branded as Puma, is a German multinational corporation that designs and manufactures athletic and casual footwear, apparel and accessories, which is headquartered in Herzogenaurach, Bavaria, Germany. Puma is the third largest sportswear manufacturer in the world."
        result_df = df[df['Brand']=="puma"][['Review','Sentiments','Polarity']]
        result_df = result_df.reset_index()[['Review','Sentiments','Polarity']]
        image = "Puma.png"
        
    return render_template("Dashboard.html",predicted_class=predicted_class,predicted_prob = predicted_prob,file=file,tables=[result_df.to_html(classes='data')],titles = result_df.columns.values,image=image,Desc = Desc)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Aug/2020 09:59:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 09:59:24] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Aug/2020 09:59:26] "POST /Dashboard HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 09:59:26] "GET /static/ HTTP/1.1" 404 -
127.0.0.1 - - [03/Aug/2020 09:59:26] "GET /styles.css HTTP/1.1" 404 -


in dashboard function
here
2.png


127.0.0.1 - - [03/Aug/2020 09:59:38] "POST /sdm HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 09:59:39] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Aug/2020 09:59:39] "GET /static/2.png HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 09:59:39] "GET /static/Nike.png HTTP/1.1" 200 -


	Nike: 99.91%
C:/Users/altha/althaf_docs/Sem 2/Social Data Mining Techniques/Final Assignment/2.png
in sdm function


In [ ]:
df = pd.read_csv("Sentiment_Analysis.csv",encoding="cp1252")
df =df.reset_index(drop=True)
df


In [ ]:
df = df.dropna()

In [ ]:
puma_df = df[df['Brand']=="puma"][['Review','Sentiments','Polarity']]
adidas_df = df[df['Brand']=="Adidas"][['Review','Sentiments','Polarity']]
nike_df = df[df['Brand']=="Nike"][['Review','Sentiments','Polarity']]

In [ ]:
adidas_df.reset_index()[['Review','Sentiments','Polarity']]